In [67]:
import os

In [68]:
%pwd #present working directory #so get inside the research folder

'd:\\'

In [69]:
#for creating artifacts go bask to project directory
os.chdir("..")

In [70]:
%pwd#so come in deep learning project

UsageError: Line magic function `%pwd#so` not found.


adding entity in modeular coding now creating inside notenook then adding.

In [71]:
# entiry is return type of any function ,returntype of data detation function
#create out custmo entity using data calss
from dataclasses import dataclass#is a decorator define top of data veriable so it can can access from anywhere this entity
from pathlib import Path

#root-dir from yaml file
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

    #here need to import two things

In [72]:
import sys
print(sys.path)

try:
    from box.exceptions import BoxValueError
    print("BoxValueError imported successfully")
except ModuleNotFoundError as e:
    print("ModuleNotFoundError:", e)


['d:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\\research', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\python38.zip', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\DLLs', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney', '', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages', 'd:\\kidney_disease_classification-deep_learning_project\\src', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages\\win32', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages\\win32\\lib', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages\\Pythonwin']
BoxValueError imported successfully


In [73]:
from box.exceptions import BoxValueError
import sys
print(sys.path)
print("Box library installed successfully")

['d:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\\research', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\python38.zip', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\DLLs', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney', '', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages', 'd:\\kidney_disease_classification-deep_learning_project\\src', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages\\win32', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages\\win32\\lib', 'c:\\Users\\HP\\Anaconda3\\envs\\kidney\\lib\\site-packages\\Pythonwin']
Box library installed successfully


In [74]:
from cnnClassifier.constants import * #* means all
from cnnClassifier.utils.common import read_yaml , create_directories

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

### this is technique used in entire project to managemetn thenentire variable 
### so get config easily by calling this config_manager

In [75]:
#inside file created constructer 2 path initialize here in this class variable
#inside this reading 2 files by read_yaml fun and store inn 2 var config and params
#create directories self.artifacts_roots it will return as key value pair

# Import constants
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

#above is configuration class inside it prepare data_ingestion related configuration
#define above in class as entiry its returntype as DataIngestionConfig liek node calss in LL 
#this returnonly value mention in DataIngestionConfig

#1.extracting data ingestion related configuration #2. and then storing one by one
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [76]:
#now update the components step firstly do in notebook then convert it to modular coding
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [77]:
#1st prepare class data ingestion it take dataingconf from configmanager
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    #that's why call pipeline 1download 2 extract
    #same as download from the google drive
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e


    #now data is dowmnload now need to extract it because it is a zip file
    #unzip location is data_ingestion
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [78]:
#define the pipeline #try block inside that #open configmanager to take data-ingestion-config
import yaml

try:
    #making objec to call the class method
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    #this will take config=dataingestion_config
    #then making object of this class to call extract and download method
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'